In [13]:
import pandas as pd
from sklearn import linear_model 
import numpy as np
import statistics

TRAIN_PATH = '../../res/ftr/base_data_train.csv'
EVALUATION_PATH = '../../res/ftr/base_data_evaluation.csv'

pd.set_option('display.max_columns', 30)

training_set = pd.read_csv(TRAIN_PATH)
evaluation_set = pd.read_csv(EVALUATION_PATH)

training_set.drop(columns=['Unnamed: 0'], inplace=True)
evaluation_set.drop(columns=['Unnamed: 0'], inplace=True)

evaluation_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,29.0,3.0,NaN,4.0,300.0,NaN,NaN,15.906,0,0,0,0,0
1,51775,NaN,1.0,1.0,1.0,67.0,67.0,113851.0,16.732,0,0,0,0,0
2,115253,0.0,2.0,1.0,2.0,87.0,100.0,23620.0,16.585,0,0,0,0,1
3,299321,2.0,2.0,2.0,2.0,86.0,86.0,129347.0,16.527,0,0,0,0,0
4,173570,10.0,2.0,1.0,1.0,80.0,76.0,57125.0,15.932,0,0,0,1,1


In [14]:
def load_features(train_df, evaluation_df, features_list):
    for feature in features_list:
        test_ftr = pd.read_csv('../../res/ftr/'+feature+'_train.csv')
        evaluation_ftr = pd.read_csv('../../res/ftr/'+feature+'_evaluation.csv')
    
        train_df = train_df.merge(test_ftr, on='id', how='inner').drop(columns=['Unnamed: 0'])
        evaluation_df = evaluation_df.merge(evaluation_ftr, on='id', how='inner').drop(columns=['Unnamed: 0'])
    
    return (train_df, evaluation_df)

(training_set, evaluation_set) = load_features(training_set, evaluation_set, ['surface_features', 'murder_rate_of_entity'])

training_set.head()

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,metrostotalesporhabitacion,metroscubiertosporhabitacion,metroscubiertossobretotales,murder_rate_of_entity
0,254099,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,16.670,0,0,0,0,0,2273000,40.000000,40.000000,1.000000,9.666667
1,53461,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,15.884,0,0,0,1,1,3600000,60.000000,89.333333,1.488889,9.666667
2,247984,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,16.725,0,0,0,0,0,1200000,55.333333,48.000000,0.867470,13.500000
3,209067,1.0,2.0,1.0,1.0,63.0,67.0,53666.0,15.408,0,0,0,1,1,650000,33.500000,31.500000,0.940299,10.333333
4,185997,10.0,2.0,1.0,1.0,95.0,95.0,47835.0,16.959,0,0,0,0,0,1150000,47.500000,47.500000,1.000000,13.500000


In [19]:
median_cols = ['antiguedad', 'habitaciones', 'banos', 'metroscubiertosporhabitacion', 'metroscubiertossobretotales']
tipo_median_cols = ['metroscubiertos', 'metrostotales']
ciudad_median_cols = ['idzona', 'murder_rate_of_entity']

median_by_tipodepropiedad = {}
mean_by_ciudad = {}

median_by_tipodepropiedad = {}
median_by_ciudad = {}

training_set['garages'] = training_set['garages'].apply(lambda x: 0 if (x == None or x == 'nan') else x)

for column in median_cols:
    median = statistics.median(training_set[column])
    print(median)
    training_set[column] = training_set[column].apply(lambda x: median if (x == None or x == 'nan') else x)

for column in training_set.columns:
    if training_set[column].isnull().values.any():
        print(column)

SyntaxError: invalid syntax (<ipython-input-19-84bf78bd1a29>, line 19)

In [ ]:
training_set_X = training_set.drop(columns=['precio']).values
training_set_Y = training_set['precio'].values

regressor = linear_model.LinearRegression(normalize = True)
regressor.fit(training_set_X, training_set_Y)